In [1]:

import numpy as np
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt 
import random 
import sys
sys.path.append('./lib')
from lib.cleaning import *
from lib.data_prepration import *
from lib.paralellism import *
from lib.mydoc2vec import *
import nltk
from nltk.corpus import stopwords
from gensim.models import Word2Vec
import nltk
import gensim
from gensim.models import Doc2Vec
from collections import namedtuple
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
#!pip install scipy==1.12
SEED = 448

[nltk_data] Downloading package stopwords to
[nltk_data]     c:\Users\alire\Downloads\neuefische\ds-
[nltk_data]     capstone\.venv\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     c:\Users\alire\Downloads\neuefische\ds-
[nltk_data]     capstone\.venv\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     c:\Users\alire\Downloads\neuefische\ds-
[nltk_data]     capstone\.venv\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:

file_path = '../data/cleared_columns.csv'
data_prep = DataPreparation(file_path)
data_prep.read_large_csv()
random.seed(SEED)


In [3]:
len(data_prep.data)

209457

In [4]:
_doc2vec_load = Doc2VecRecommender(data_prep.data)
_doc2vec_load.load_model('./models/doc2vec_model_hole')
#_doc2vec_load.train_similarity()

In [5]:
vecs=_doc2vec_load.get_all_vecs()

random_indices = np.random.choice(len(vecs), size=10000, replace=False)
sampled_data = vecs[random_indices,:]
vecs.shape


KeyboardInterrupt: 

In [ ]:

silhouette_coefficients = []

kmeans_kwargs = {
    "init": "k-means++",
    "n_init": 10,
    "max_iter": 100,
    "random_state": 42,}

for k in range(20, 250,5):
    kmeans = KMeans(n_clusters=k, **kmeans_kwargs)
    kmeans.fit(sampled_data)
    score = silhouette_score(sampled_data, kmeans.labels_)
    silhouette_coefficients.append(score)
    print(f"{k}: {score}")


In [ ]:
num_clusters = 200


kmeans = KMeans(n_clusters=num_clusters,init="k-means++",n_init="auto",max_iter=200)

kmeans.fit(vecs)

clusters = kmeans.labels_.tolist()



In [ ]:
"""score = silhouette_score(vecs, kmeans.labels_)
score"""
len(clusters)
data_cluster=data_prep.data.copy()
data_cluster=data_cluster[['title', 'price', 'helpfulness', 'score', 'description', 'authors', 'publisher', 'categories', 'ratingscount','all_clear']]
data_cluster['clusters']=clusters
data_cluster.data.to_csv('../data/cleared_columns_clusters.csv')


In [ ]:
np.array(vecs).shape

In [ ]:
search='harry potter'
indexs=_doc2vec_load.get_similars_by_gensim(search,vecs)
data_prep.data.loc[indexs].drop_duplicates(subset='title')[['title']]

In [ ]:

_doc2vec_load.recommend_by_text([search])

In [ ]:
indexs=_doc2vec_load.get_similars(search)
data_prep.data.loc[indexs[0]]